In [1]:
import numpy as np
import pandas as pd

C:\Users\kasLe\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
movies.shape

(4803, 20)

In [5]:
credits.shape

(4803, 4)

In [6]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


The movies and credits dataframes contain information about the same set of movies. To simplify, we merge the two dataframes into a single one. We can perform the merge either using the movie_id or the title column. In this case, we are merging based on the title column.

In [7]:
movies = movies.merge(credits,on='title')

In [8]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

To build a content-based recommender system, we need to decide which columns to retain in the dataset. Our goal is to create meaningful tags for each movie, so we will focus on the columns that help in generating these tags.

- **Budget**: This is not useful for creating tags. Liking one high-budget movie doesn't necessarily mean you'll like another just because of its budget.
  
- **Genre**: This is important. People often enjoy movies from certain genres, like romance, making it a key factor in tag generation.
  
- **Homepage**: Not important. This doesn't contribute to movie similarity.
  
- **Keywords**: Crucial, as these serve as direct tags that help in identifying similarities between movies.
  
- **Original Language**: Not helpful. Out of 4803 movies, 4510 are in English, so this column is highly imbalanced and unlikely to offer useful differentiation.
  
- **Title**: Necessary for identifying the movie, but we'll remove original_title as it’s redundant.
  
- **Overview**: Very important. By comparing movie overviews, we can determine content similarity, making it a critical feature for generating tags.
  
- **Popularity**: Could be useful, but it's a numerical value that doesn't align with our tag-based approach.
  
- **Production Companies**: While it could be relevant, recommending movies based on production companies is less common.
  
- **Production Countries**: Not relevant. It doesn’t help with recommending movies.
  
- **Release Date**: May be of some interest (e.g., people may prefer movies from a certain time period like the early 2000s), but it’s a numerical value and doesn't fit into our current tag-generation approach.
  
- **Revenue**: Similar to budget, it’s a numerical value that doesn’t contribute to our tag-based methodology.
  
- **Runtime**: Not useful for generating tags.
  
- **Spoken Languages**: Not relevant to movie recommendations.
  
- **Status**: Not important for generating movie tags.
  
- **Tagline**: While taglines can provide insight, the overview column already serves the same purpose, so we don't need to include it.
  
- **Vote Average**: Numerical and doesn't align with our approach.
  
- **Vote Count**: Also numerical, so it's not part of our tag-based system.
  
- **Cast**: Very important. Movie recommendations are often based on actors, making this a key feature.
  
- **Crew**: Important. Movie recommendations are often driven by directors and other crew members.

So, the columns we’ll use to create tags for each movie are: **genres**, **id**, **keywords**, **title**, **overview**, **cast**, and **crew**.


In [10]:
movies['original_language'].value_counts()

original_language
en    4510
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
ko      12
cn      12
ru      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: count, dtype: int64

In [11]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [12]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [13]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [14]:
movies.dropna(inplace=True) 
# drops rows that contain missing values (NaN) in any column 
# of the movies DataFrame

In [15]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [16]:
movies.duplicated().sum()

0

=> no duplicate rows

In [17]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [18]:
import ast
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [19]:
movies['genres'] = movies['genres'].apply(convert)

In [20]:
movies['keywords'] = movies['keywords'].apply(convert)

In [21]:
def convert3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter+=1
        else:
            break
    return l
movies['cast'] = movies['cast'].apply(convert3)

In [22]:
def convert_crew(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == "Director":
            l.append(i['name'])
            break
    return l
movies['crew'] = movies['crew'].apply(convert_crew)

In [23]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [24]:
# overview column is a string
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [25]:
# we'll convert overview into a list so we can concatenate with the other lists
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [26]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


'Sam Worthington' => 'SamWorthington'

This transformation is necessary to avoid confusion between similar names.
Without removing the space, the model could mistakenly associate 'Sam Worthington' with 'Sam Mendes', leading to incorrect recommendations if the user specifically wants movies with Sam Worthington.
By concatenating the first and last names into 'SamWorthington', we ensure that the model treats it as a unique entity, eliminating any ambiguity between different individuals with similar names.

In [28]:
movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])

0       [Action, Adventure, Fantasy, ScienceFiction]
1                       [Adventure, Fantasy, Action]
2                         [Action, Adventure, Crime]
3                   [Action, Crime, Drama, Thriller]
4                [Action, Adventure, ScienceFiction]
                            ...                     
4804                       [Action, Crime, Thriller]
4805                               [Comedy, Romance]
4806               [Comedy, Drama, Romance, TVMovie]
4807                                              []
4808                                   [Documentary]
Name: genres, Length: 4806, dtype: object

In [29]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [30]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [31]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [32]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [33]:
new_df = movies[['movie_id', 'title', 'tags']]

In [34]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [35]:
new_df[new_df['title'] == 'Spectre'].index[0]

2

In [36]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\kasLe\AppData\Local\Temp\ipykernel_25976\3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [37]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [38]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [39]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

C:\Users\kasLe\AppData\Local\Temp\ipykernel_25976\3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [40]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


## Problem Statement

The goal is to recommend the 5 most similar movies when the user enters the name of a movie. To achieve this, we calculate the similarity score between the tags of the entered movie and all other movies. 

### Approach to Calculating Similarity

One way to calculate similarity is by comparing the number of common words between the movie tags. Tags with more matching words will have a higher similarity score. Another way to approach this is through text vectorization.

### Text Vectorization Techniques

There are several techniques for text vectorization, including:
- Bag of Words
- TF-IDF
- Word2Vec

For simplicity, we use **Bag of Words**.

### Bag of Words Approach

1. **Combine all words from movie tags**: 
   - Create a large string containing all the words from the movie tags.
   - Identify the 5000 most common words in this combined string (words with the greatest frequency).

2. **Create a matrix**:
   - For each movie, count the occurrences of each of the 5000 most frequent words in the movie's tags.
   - This results in a table where each row represents a movie and each column corresponds to a frequent word.

   Example table:

         word1   word2   word3  ..   word5000
 movie1    5       3       4            0
 movie2    1       2       0            0
 movie3    0       6       1            1
 ...
 movie4000
 
 3. **Vector representation**:
- Each row in the above table is a vector representing a movie.
- If there were only two words (word1 and word2), this would be a 2D vector space with 4000 movie vectors plotted on the x and y axes.
- With 5000 words, each movie is represented as a vector in a 5000-dimensional space.

### Choosing the Number of Most Common Words

The number of most common words (5000 in this case) should be chosen carefully. Using more words increases the dimensionality of the vector space, which can reduce performance. The goal is to balance performance and dimensionality.

### Excluding Stop Words

Stop words (e.g., "a", "the", "and", "of", "to") don't contribute to the meaning of the tags, so they should not be considered during vectorization. These words are used for sentence structure but don’t impact the content or similarity.

### Automating Vectorization

While vectorization can be done manually, libraries like **CountVectorizer** in Scikit-learn can automate the process efficiently.

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words='english') # max_features = how many most frequent words

In [42]:
vectors = cv.fit_transform(new_df['tags']).toarray() #cv returns a scipy sparse matrix, we explicitly convert it into a numpy array

In [43]:
vectors.shape

(4806, 5000)

In [44]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [45]:
vectors[0] #vector of first movie

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [46]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

### Issue with Word Variations

In the previous approach, we might encounter issues where words with the same meaning are treated as separate features in our data. For example, the words **"activity"** and **"activities"** refer to the same concept in the context of movies, but they would be treated as distinct words in the corpus.

### Solution: Stemming

To solve this problem, we can apply **stemming**, a technique that reduces words to their base or root form. This way, variations of a word (e.g., "loved", "loving", "love") are all converted to the same root form.

#### Example:

- Input: `['loved', 'loving', 'love']`
- After Stemming: `['love', 'love', 'love']`

By stemming words, we ensure that different forms of a word are treated as a single feature, improving the accuracy of the model and reducing unnecessary dimensionality.

In [47]:
# !pip install nltk
import nltk

In [48]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [49]:
ps.stem('loving')

'love'

In [50]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    string = " ".join(y)
    return string

In [51]:
stem('In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron')

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [52]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\kasLe\AppData\Local\Temp\ipykernel_25976\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [53]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [54]:
cv_1 = CountVectorizer(max_features = 5000, stop_words='english')
vectors_1 = cv_1.fit_transform(new_df['tags']).toarray()

In [55]:
cv_1.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

### Calculating Similarity Between Movie Vectors

Once we have vectorized the movie tags, the next step is to calculate the similarity between each movie vector and every other movie vector. The greater the distance between two vectors, the lower the similarity between the corresponding movies.

#### Euclidean Distance vs. Cosine Distance

Instead of using **Euclidean distance** (which measures the straight-line distance between two points, i.e., `sqrt((x2 - x1)^2 + (y2 - y1)^2)`), we will use **Cosine similarity** (which measures the angle between two vectors).

- **Cosine Similarity**: This measures the angle between two vectors. The smaller the angle (theta), the closer the vectors are, meaning the movies are more similar.
    - **0°**: Identical vectors (high similarity)
    - **5°**: Vectors are quite close (moderate similarity)
    - **90°**: Vectors are perpendicular (very dissimilar)

#### Why Cosine Distance Instead of Euclidean Distance?

Euclidean distance is not ideal for high-dimensional data (such as our 5000-dimensional movie vectors). As the number of dimensions increases, Euclidean distance becomes less reliable because it tends to underestimate the distance between vectors in high-dimensional spaces. Cosine similarity, on the other hand, is better suited for this scenario, as it focuses on the angle between the vectors rather than their absolute position in space.

In [56]:
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
similarity = cosine_similarity(vectors_1) #similarity score of every movie with every other movie

In [58]:
similarity

array([[1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
        0.        ],
       [0.08346223, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.0860309 , 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]])

In [59]:
similarity.shape

(4806, 4806)

In [60]:
similarity[0] # similarity of first movie with every other movie

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

 the diagonal of the 2d similarity array is always 1, its similarity of each movie with itself.

In [61]:
new_df[new_df['title'] == 'Batman Begins'].index[0]

119

In [62]:
similarity[0]

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

### Maintaining Movie Indexes After Sorting

When we sort the similarity array, we lose track of which similarity corresponds to which movie. For example, if the similarity between the first and second movies is `0.08346223`, after sorting, this value will move to a different position, and we will no longer know that it's the similarity between the first and second movies.

#### Solution: Using `enumerate`

To solve this issue, we can use the `enumerate()` function. This allows us to retain the original index of each movie along with its similarity score, even after sorting. Here's how it works:

- `enumerate()` adds a counter (the original index) to each element in the similarity list.
- After sorting, we can still refer to the original index of each movie.

This way, even when the similarity values are reordered, we can easily identify which movie each similarity score corresponds to.

In [86]:
# list(enumerate(similarity[0]))

In [87]:
# sorted(list(enumerate(similarity[0])), reverse=True, key= lambda x:x[1])

In [65]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key= lambda x:x[1])[1:6]
    for i in movies_list:
        print(i[0])
        print(new_df.iloc[i[0]].title)

In [66]:
new_df.iloc[0].title

'Avatar'

In [67]:
recommend('Avatar')

1214
Aliens vs Predator: Requiem
2405
Aliens
3728
Falcon Rising
507
Independence Day
539
Titan A.E.


In [85]:
recommendations = []
for movie_index in range(len(new_df)):
    distances = similarity[movie_index]
    movies_list = sorted(
        list(enumerate(distances)), reverse=True, key=lambda x: x[1]
    )[1:11]  # Get top 10 similar movies (excluding the movie itself)

    # Store recommendations as (movie index, similarity score)
    recommendations.append({
        'movie': new_df.iloc[movie_index].title,
        'recommendations': [
            {'title': new_df.iloc[i[0]].title, 'score': i[1]} for i in movies_list
        ]
    })

# Dump recommendations to pickle files

pickle.dump(recommendations, open('recommendations.pkl', 'wb'))

In [68]:
import pickle

In [69]:
pickle.dump(new_df, open('movies.pkl', 'wb'))

In [70]:
new_df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [71]:
pickle.dump(similarity,open('similarity.pkl', 'wb'))

In [72]:
new_df_sm = new_df.sample(frac=0.1, random_state=42).reset_index(drop=True)

In [73]:
new_df_sm.shape

(481, 3)

In [74]:
new_df_sm.head()

,movie_id,title,tags
0,8967,The Tree of Life,the impressionist stori of a texa famili in th...
1,12837,The Secret Life of Bees,"set in south carolina in 1964, thi is the tale..."
2,1370,Rambo III,"combat ha taken it toll on rambo, but he' fina..."
3,593,Solaris,ground control ha been receiv strang transmiss...
4,767,Harry Potter and the Half-Blood Prince,"as harri begin hi sixth year at hogwarts, he d..."


In [75]:
cv_sm = CountVectorizer(max_features = 5000, stop_words='english')
vectors_sm = cv_sm.fit_transform(new_df_sm['tags']).toarray()

In [76]:
similarity_sm = cosine_similarity(vectors_sm)

In [77]:
similarity_sm

array([[1.        , 0.09731237, 0.03615508, ..., 0.08703883, 0.20519567,
        0.12734291],
       [0.09731237, 1.        , 0.06333005, ..., 0.08131156, 0.01996808,
        0.04956816],
       [0.03615508, 0.06333005, 1.        , ..., 0.09440686, 0.05564149,
        0.06906139],
       ...,
       [0.08703883, 0.08131156, 0.09440686, ..., 1.        , 0.12501993,
        0.15517288],
       [0.20519567, 0.01996808, 0.05564149, ..., 0.12501993, 1.        ,
        0.17420142],
       [0.12734291, 0.04956816, 0.06906139, ..., 0.15517288, 0.17420142,
        1.        ]])

In [78]:
similarity_sm.shape

(481, 481)

In [79]:
def recommend_sm(movie):
    movie_index = new_df_sm[new_df_sm['title'] == movie].index[0]
    distances = similarity_sm[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key= lambda x:x[1])[1:6]
    for i in movies_list:
        print(i[0])
        print(new_df_sm.iloc[i[0]].title)

In [80]:
new_df_sm.iloc[0].title

'The Tree of Life'

In [81]:
recommend_sm('The Tree of Life')

287
Hesher
190
Juliet and Alfa Romeo
231
Memoirs of an Invisible Man
39
The Young Messiah
163
There Be Dragons


In [82]:
import pickle

In [83]:
pickle.dump(new_df_sm, open('movies_sm.pkl', 'wb'))

In [84]:
pickle.dump(similarity_sm,open('similarity_sm.pkl', 'wb'))